In [ ]:
import random
import math
import time
import re

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

# Auxiliar functions

In [5]:
def parse_vrp(path):
    coords = {}
    demands = {}
    capacity = None
    depot = 1
    section = None
    with open(path, 'r', encoding='utf-8', errors='ignore') as f:
        for raw in f:
            line = raw.strip()
            if not line:
                continue
            up = line.upper()
            if 'CAPACITY' in up and ':' in line:
                try:
                    capacity = int(line.split(':')[1])
                except:
                    for p in line.split():
                        if p.isdigit():
                            capacity = int(p); break

            if up.startswith('NODE_COORD_SECTION'):
                section = 'coords'
                continue
            if up.startswith('DEMAND_SECTION'):
                section = 'demands'
                continue
            if up.startswith('DEPOT_SECTION'):
                section = 'depot'
                continue

            if section == 'coords':
                parts = line.split()
                if len(parts) >= 3:
                    idx = int(parts[0])
                    x = float(parts[1])
                    y = float(parts[2])
                    coords[idx] = (x,y)

            elif section == 'demands':
                parts = line.split()
                if len(parts) >= 2:
                    idx = int(parts[0])
                    q = float(parts[1])
                    demands[idx] = q

            elif section == 'depot':
                if line.startswith("-1"):
                    continue
                try:
                    depot = int(line)
                except:
                    pass

    if depot not in demands:
        demands[depot] = 0.0
    for k in coords:
        if k not in demands:
            demands[k] = 0.0

    if capacity is None:
        raise ValueError("VRP file missing CAPACITY")

    return coords, demands, depot, int(capacity)


def euclid(a,b):
    return math.hypot(a[0]-b[0], a[1]-b[1])

def build_distance_matrix(coords):
    ids = sorted(coords.keys())
    id_to_idx = {idv:i for i,idv in enumerate(ids)}
    n = len(ids)
    mat = np.zeros((n,n))
    for i,id1 in enumerate(ids):
        for j,id2 in enumerate(ids):
            mat[i,j] = euclid(coords[id1], coords[id2])
    return mat, ids, id_to_idx


def route_cost(route, dist_mat, id_to_idx):
    total = 0
    for i in range(len(route)-1):
        total += dist_mat[id_to_idx[route[i]], id_to_idx[route[i+1]]]
    return total

def total_cost(routes, dist_mat, id_to_idx):
    return sum(route_cost(r, dist_mat, id_to_idx) for r in routes)




# GRASP

In [ ]:
def greedy_randomized_construction(coords, demands, depot, capacity, dist_mat, ids, id_to_idx,
                                   alpha=0.2, perturb=False, perturb_scale=0.1):

    unvisited = set(k for k in ids if k != depot)
    routes = []

    while unvisited:
        route = [depot]
        load = 0
        cur = depot

        while True:
            candidates = []
            for c in unvisited:
                if load + demands[c] <= capacity:
                    d = dist_mat[id_to_idx[cur], id_to_idx[c]]
                    if perturb:
                        factor = 1 + random.uniform(-perturb_scale, perturb_scale)
                        dsel = d * factor
                    else:
                        dsel = d
                    candidates.append((c, dsel))

            if not candidates:
                break

            candidates.sort(key=lambda x: x[1])
            rcl_size = max(1, int(len(candidates)*alpha))
            chosen = random.choice(candidates[:rcl_size])[0]

            route.append(chosen)
            load += demands[chosen]
            unvisited.remove(chosen)
            cur = chosen

        route.append(depot)
        routes.append(route)

    return routes

def intelligent_insertion_construction(coords, demands, depot, capacity, dist_mat, ids, id_to_idx,
                                       alpha=0.2, perturb=False, perturb_scale=0.1):

    unvisited = set(k for k in ids if k != depot)
    routes = []

    while unvisited:
        insertions = []

        for c in unvisited:
            dem = demands[c]

            for r_idx, r in enumerate(routes):
                load = sum(demands[n] for n in r if n != depot)
                if load + dem > capacity:
                    continue

                for pos in range(1, len(r)):
                    a = r[pos-1]
                    b = r[pos]
                    inc = (dist_mat[id_to_idx[a], id_to_idx[c]] +
                           dist_mat[id_to_idx[c], id_to_idx[b]] -
                           dist_mat[id_to_idx[a], id_to_idx[b]])

                    if perturb:
                        inc_sel = inc * (1 + random.uniform(-perturb_scale, perturb_scale))
                    else:
                        inc_sel = inc

                    insertions.append((c, r_idx, pos, inc_sel))

            if dem <= capacity:
                base = dist_mat[id_to_idx[depot], id_to_idx[c]] * 2
                if perturb:
                    base_sel = base * (1 + random.uniform(-perturb_scale, perturb_scale))
                else:
                    base_sel = base
                insertions.append((c, None, None, base_sel))

        if not insertions:
            for c in list(unvisited):
                if demands[c] <= capacity:
                    routes.append([depot, c, depot])
                    unvisited.remove(c)
            break

        insertions.sort(key=lambda x: x[3])
        rcl_size = max(1, int(len(insertions)*alpha))
        c, r_idx, pos, _ = random.choice(insertions[:rcl_size])

        if r_idx is None:
            routes.append([depot, c, depot])
        else:
            routes[r_idx].insert(pos, c)

        unvisited.remove(c)

    return routes

def two_opt(route, dist_mat, id_to_idx):
    improved = True
    best = route
    best_cost = route_cost(best, dist_mat, id_to_idx)

    while improved:
        improved = False
        n = len(best)

        for i in range(1, n-2):
            for j in range(i+1, n-1):
                if j == i+1:
                    continue

                new_r = best[:i] + best[i:j+1][::-1] + best[j+1:]
                c_new = route_cost(new_r, dist_mat, id_to_idx)

                if c_new < best_cost - 1e-9:
                    best = new_r
                    best_cost = c_new
                    improved = True
                    break
            if improved:
                break

    return best

def relocate(routes, demands, capacity, dist_mat, id_to_idx):
    improved = False
    best_gain = 0
    best_move = None

    loads = [sum(demands[n] for n in r if n != r[0] and n != r[-1]) for r in routes]

    for i, r_from in enumerate(routes):
        for pos in range(1, len(r_from)-1):
            c = r_from[pos]

            for j, r_to in enumerate(routes):
                if i == j:
                    continue

                if loads[j] + demands[c] > capacity:
                    continue

                for pos_to in range(1, len(r_to)):
                    a = r_from[pos-1]
                    b = r_from[pos+1]
                    delta_remove = -(dist_mat[id_to_idx[a], id_to_idx[c]]
                                     + dist_mat[id_to_idx[c], id_to_idx[b]]
                                     - dist_mat[id_to_idx[a], id_to_idx[b]])

                    x = r_to[pos_to-1]
                    y = r_to[pos_to]
                    delta_insert = (dist_mat[id_to_idx[x], id_to_idx[c]]
                                    + dist_mat[id_to_idx[c], id_to_idx[y]]
                                    - dist_mat[id_to_idx[x], id_to_idx[y]])

                    gain = -(delta_remove + delta_insert)

                    if gain > best_gain + 1e-9:
                        best_gain = gain
                        best_move = (i, pos, j, pos_to, c)

    if best_move:
        i, pos, j, pos_to, c = best_move
        routes[j].insert(pos_to, c)
        del routes[i][pos]
        improved = True

    return improved

def improve_solution(routes, demands, capacity, dist_mat, id_to_idx, max_no_improv=5):
    no_improv = 0
    best_cost = total_cost(routes, dist_mat, id_to_idx)

    while no_improv < max_no_improv:
        improved = False

        for idx, r in enumerate(routes):
            newr = two_opt(r, dist_mat, id_to_idx)
            if route_cost(newr, dist_mat, id_to_idx) < route_cost(r, dist_mat, id_to_idx) - 1e-9:
                routes[idx] = newr
                improved = True

        if relocate(routes, demands, capacity, dist_mat, id_to_idx):
            improved = True

        new_cost = total_cost(routes, dist_mat, id_to_idx)

        if new_cost < best_cost - 1e-9:
            best_cost = new_cost
            no_improv = 0
        else:
            no_improv += 1

        if not improved:
            no_improv += 1

    return routes

def shake_solution(routes, strength=1):
    customers = [c for r in routes for c in r if c != r[0] and c != r[-1]]
    if not customers:
        return routes

    for _ in range(strength):
        c = random.choice(customers)

        ri, pos = None, None
        for i,r in enumerate(routes):
            if c in r:
                ri = i
                pos = r.index(c)
                break

        choices = list(range(len(routes)))
        choices.remove(ri)

        if not choices:
            routes.append([routes[0][0], c, routes[0][0]])
            del routes[ri][pos]
        else:
            rj = random.choice(choices)
            pos_to = random.randint(1, len(routes[rj])-1)
            routes[rj].insert(pos_to, c)
            del routes[ri][pos]

    routes = [r for r in routes if len(r) > 2]
    return routes


def run_grasp(instance_path, coords, demands, depot, capacity,
              dist_mat, ids, id_to_idx,
              max_iters=100, alpha=0.25,
              perturb=False, intelligent=False,
              intensify=True, diversify=True,
              time_limit=None, seed=None, method_name="method"):

    if seed is not None:
        random.seed(seed)
        np.random.seed(seed)

    logs = []
    best_solution = None
    best_cost = float('inf')

    start_time = time.time()

    for it in range(1, max_iters+1):
        if time_limit and time.time() - start_time > time_limit:
            break

        t0 = time.time()

        if intelligent:
            routes = intelligent_insertion_construction(
                coords, demands, depot, capacity,
                dist_mat, ids, id_to_idx,
                alpha=alpha, perturb=perturb)
        else:
            routes = greedy_randomized_construction(
                coords, demands, depot, capacity,
                dist_mat, ids, id_to_idx,
                alpha=alpha, perturb=perturb)

        constructed_cost = total_cost(routes, dist_mat, id_to_idx)

        if intensify:
            routes = improve_solution(routes, demands, capacity,
                                      dist_mat, id_to_idx)

        improved_cost = total_cost(routes, dist_mat, id_to_idx)

        t_iter = time.time() - t0
        total_elapsed = time.time() - start_time

        logs.append({
            "iteration": it,
            "time_iter": t_iter,
            "time_total": total_elapsed,
            "constructed_cost": constructed_cost,
            "improved_cost": improved_cost,
            "num_routes": len(routes),
            "route_costs": ";".join(f"{route_cost(r, dist_mat, id_to_idx):.4f}" for r in routes),
            "instance": instance_path.split('/')[-1].replace(".vrp",""),
            "method": method_name
        })

        if improved_cost < best_cost - 1e-9:
            best_cost = improved_cost
            best_solution = [list(r) for r in routes]

        if diversify and it % max(1, int(max_iters*0.1)) == 0:
            if best_solution:
                shake_solution([list(r) for r in best_solution])

    df = pd.DataFrame(logs)
    return df, best_solution, best_cost




# Experimets

In [ ]:

def run_experiments(instance_paths, outdir=".", max_iters=200,
                    alpha=0.25, time_limit_per_config=60, seed=42):

    configs = [
        ("grasp_standard", False, False),
        ("grasp_cost_perturb", True, False),
        ("grasp_intelligent", False, True),
        ("grasp_full", True, True)
    ]

    all_logs = []

    for inst_path in instance_paths:
        coords, demands, depot, capacity = parse_vrp(inst_path)
        dist_mat, ids, id_to_idx = build_distance_matrix(coords)

        for name, perturb, intel in configs:
            df_log, best_sol, best_cost = run_grasp(
                inst_path, coords, demands, depot, capacity,
                dist_mat, ids, id_to_idx,
                max_iters=max_iters,
                alpha=alpha,
                perturb=perturb,
                intelligent=intel,
                intensify=True,
                diversify=True,
                time_limit=time_limit_per_config,
                seed=seed,
                method_name=name
            )

            all_logs.append(df_log)

    final_df = pd.concat(all_logs, ignore_index=True)
    final_path = f"{outdir}/ALL_RESULTS.csv"
    final_df.to_csv(final_path, index=False)

    print("Saved:", final_path)
    return final_df

In [10]:
instances = ["instance1.vrp","instance2.vrp","instance3.vrp","instance4.vrp","instance5.vrp","instance6.vrp","instance7.vrp","instance8.vrp"]

run_experiments(
    instances,
    outdir=".",
    max_iters=100,
    alpha=0.25,
    time_limit_per_config=60*30,
    seed=42
)


Saved: ./ALL_RESULTS.csv


,iteration,time_iter,time_total,constructed_cost,improved_cost,num_routes,route_costs,instance,method
0,1,0.073306,0.073309,43920.855399,32766.010879,27,671.4731;641.2653;518.1434;792.0467;1322.9116;...,instance1,grasp_standard
1,2,0.065015,0.138370,46660.386354,32800.655462,27,813.1115;674.9969;809.1486;781.3350;735.0942;6...,instance1,grasp_standard
2,3,0.055961,0.194365,45109.145746,33153.234368,27,532.1732;710.0210;998.1485;652.0631;782.5296;1...,instance1,grasp_standard
3,4,0.057635,0.252040,45417.729270,33351.698423,27,661.5279;942.1772;865.1842;1789.0220;742.7442;...,instance1,grasp_standard
4,5,0.069997,0.322070,47684.596861,32545.233045,27,1004.0325;1141.0647;488.1855;1305.1553;647.740...,instance1,grasp_standard
...,...,...,...,...,...,...,...,...,...
3073,96,5.993502,629.394384,234378.490756,150194.101615,40,1920.6975;3026.4006;3731.0497;4317.8064;3148.5...,instance8,grasp_full
3074,97,6.223927,635.618408,236266.870363,151496.121357,40,2848.1581;3046.7917;2291.7494;2651.7063;3132.1...,instance8,grasp_full
3075,98,6.066136,641.684641,231468.401849,148925.286271,40,2736.5576;2584.4694;3480.7932;3727.8075;3441.9...,instance8,grasp_full
3076,99,6.067769,647.752507,232871.673314,153111.293546,40,2714.1584;3632.0020;3138.2356;3792.1495;3980.1...,instance8,grasp_full


# Graphs

In [94]:

sns.set_theme(style="whitegrid", context="talk")
plt.rcParams['font.family'] = 'sans-serif'

METHOD_COLORS = {
    "GRASP Standard": "#1f77b4",      
    "GRASP Cost Perturb": "#ff7f0e",  
    "GRASP Intelligent": "#2ca02c",   
    "Grasp Cost Perturb + Intelligent": "#d62728",
    "Savings Heuristic": "#9467bd", 
    "Regret Insertion Heuristic": "#8c564b",
    "Sweep Algorithm": "#e377c2"       
}


df = pd.read_csv("combined_cvrp_report.csv")
def normalize_inst_new(name):
    num = int(re.findall(r"\d+", str(name))[0])
    mapped = ((num - 1) % 8) + 1
    return f"instance{mapped}"

df["instance"] = df["instance"].apply(normalize_inst_new)


for col in df.select_dtypes(include="object").columns:
    df[col] = (
        df[col]
        .str.replace("_", " ", regex=False)
        .str.title()
    )

instances = sorted(df["instance"].unique())
methods = sorted(df["method"].unique())

plt.figure(figsize=(14, 8))

for method in methods:
    dfm = df[df["method"] == method]
    
    instance_costs = dfm.groupby("instance")["total_cost"].mean()
    
    color = METHOD_COLORS.get(method, "#333333")
    
    plt.plot(
        instances,
        [instance_costs.get(inst, float('nan')) for inst in instances],
        color=color,
        linewidth=2.5,
        marker='o',
        markersize=6,
        label=method
    )

plt.xlabel("Instance", fontsize=14)
plt.ylabel("Total Cost", fontsize=14)
plt.title("Heuristics Comparison", fontsize=18, fontweight='bold')

plt.xticks(rotation=45, ha='right')

plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')  
plt.grid(True, alpha=0.3)

sns.despine()

plt.tight_layout()

filename = "all_methods_comparison.png"
plt.savefig(filename, dpi=100, bbox_inches='tight')
plt.close()

print(f"Saved: {filename}")

Saved: all_methods_comparison.png


In [95]:
sns.set_theme(style="whitegrid", context="talk")
plt.rcParams['font.family'] = 'sans-serif'

METHOD_COLORS = {
    "Grasp Standard": "#1f77b4",       
    "Grasp Cost Perturb": "#ff7f0e",   
    "Grasp Intelligent": "#2ca02c",    
    "Grasp Cost Perturb + Intelligent": "#d62728"
}

df = pd.read_csv("ALL_RESULTS.csv")
df['method'] = df['method'].replace('grasp_full', 'Grasp Cost Perturb + Intelligent')

for col in df.select_dtypes(include="object").columns:
    df[col] = (
        df[col]
        .str.replace("_", " ", regex=False)
        .str.title()
    )

instances = sorted(df["instance"].unique())
methods = sorted(df["method"].unique())

for inst in instances:
    dfi = df[df["instance"] == inst]

    y_min = dfi["improved_cost"].min()
    y_max = dfi["improved_cost"].max()
    
    y_range = y_max - y_min
    y_lim_bottom = y_min - (y_range * 0.05)
    y_lim_top = y_max + (y_range * 0.05)

    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    axes = axes.flatten()

    for i, method in enumerate(methods):
        ax = axes[i]
        dfm = dfi[dfi["method"] == method]

        color = METHOD_COLORS.get(method, "#333333")

        ax.plot(
            dfm["iteration"],
            dfm["improved_cost"],
            color=color,
            linewidth=2.5,
            alpha=0.9,
            label=method
        )

        ax.set_title(method, fontsize=16, fontweight='bold', color='#333333')
        ax.set_xlabel("Iteration", fontsize=12)
        ax.set_ylabel("Cost", fontsize=12)
        
        ax.set_ylim(y_lim_bottom, y_lim_top)
        
        sns.despine(ax=ax)

    for j in range(len(methods), len(axes)):
        axes[j].axis("off")

    plt.suptitle(f"Instance: {inst}", fontsize=20, fontweight='bold', y=0.98)
    plt.tight_layout(rect=[0, 0, 1, 0.96])

    filename = f"{inst}_METHODS_4PLOTS.png"
    plt.savefig(filename, dpi=100, bbox_inches='tight')
    plt.close()

    print(f"Saved: {filename}")

Saved: Instance1_METHODS_4PLOTS.png
Saved: Instance2_METHODS_4PLOTS.png
Saved: Instance3_METHODS_4PLOTS.png
Saved: Instance4_METHODS_4PLOTS.png
Saved: Instance5_METHODS_4PLOTS.png
Saved: Instance6_METHODS_4PLOTS.png
Saved: Instance7_METHODS_4PLOTS.png
Saved: Instance8_METHODS_4PLOTS.png


In [96]:
sns.set_theme(style="whitegrid", context="talk")

METHOD_COLORS = {
    "GRASP Standard": "#1f77b4",       
    "GRASP Cost Perturb": "#ff7f0e",   
    "GRASP Intelligent": "#2ca02c",    
    "Grasp Cost Perturb + Intelligent": "#d62728",
    "Savings Heuristic": "#9467bd",    
    "Regret Insertion Heuristic": "#8c564b",
    "Sweep Algorithm": "#e377c2"       
}

df_grasp = pd.read_csv("ALL_RESULTS.csv")
df_grasp['method'] = df_grasp['method'].replace('grasp_full', 'Grasp Cost Perturb + Intelligent')

df_heur = pd.read_csv("combined_cvrp_report.csv")

def normalize_inst_new(name):
    num = int(re.findall(r"\d+", str(name))[0])
    mapped = ((num - 1) % 8) + 1
    return f"instance{mapped}"

df_heur["instance_norm"] = df_heur["instance"].apply(normalize_inst_new)

def normalize_inst_grasp(name):
    num = re.findall(r"\d+", str(name))
    if num:
        return f"instance{int(num[0])}"
    return str(name).lower()

df_grasp["instance_norm"] = df_grasp["instance"].apply(normalize_inst_grasp)

best_grasp = (
    df_grasp
    .groupby(["instance_norm", "method"])["improved_cost"]
    .min()
    .reset_index()
    .rename(columns={"improved_cost": "best_cost"})
)

method_name_map = {
    "grasp_standard": "GRASP Standard",
    "grasp_cost_perturb": "GRASP Cost Perturb",
    "grasp_intelligent": "GRASP Intelligent",
    "Grasp Cost Perturb + Intelligent": "Grasp Cost Perturb + Intelligent",  
}
best_grasp["method"] = best_grasp["method"].map(method_name_map)

df_heur.rename(columns={"total_cost": "best_cost"}, inplace=True)
df_heur["method"] = df_heur["method"].str.replace("_", " ").str.title()

df_all = pd.concat([
    best_grasp[["instance_norm", "method", "best_cost"]],
    df_heur[["instance_norm", "method", "best_cost"]]
], ignore_index=True)

methods_order = [
    "GRASP Standard", "GRASP Cost Perturb", "GRASP Intelligent", "Grasp Cost Perturb + Intelligent",
    "Savings Heuristic", "Regret Insertion Heuristic", "Sweep Algorithm",
]

instances = sorted(df_all["instance_norm"].unique())
for inst in instances:
    dfi = df_all[df_all["instance_norm"] == inst]
    
    dfi = dfi.set_index("method").reindex(methods_order).dropna().reset_index()

    bar_colors = [METHOD_COLORS.get(m, "#333333") for m in dfi["method"]]

    plt.figure(figsize=(14, 7)) 
    
    bars = plt.bar(
        dfi["method"], 
        dfi["best_cost"], 
        color=bar_colors, 
        edgecolor="white", 
        linewidth=1,
        alpha=0.9
    )

    for bar in bars:
        height = bar.get_height()
        plt.text(
            bar.get_x() + bar.get_width()/2., 
            height, 
            f'{int(height)}', 
            ha='center', va='bottom', fontsize=11, fontweight='bold', color='#444444'
        )

    plt.title(f"Instance: {inst}", fontsize=20, fontweight='bold', pad=20)
    plt.ylabel("Cost", fontsize=14)
    
    plt.xticks(rotation=30, ha='right', fontsize=12)
    
    sns.despine(left=True)
    plt.grid(axis='y', alpha=0.4, linestyle='--') 
    plt.tick_params(axis='y', length=0) 

    filename = f"COMPARE_{inst}.png"
    plt.tight_layout()
    plt.savefig(filename, dpi=100)
    plt.close()

    print(f"Saved: {filename}")

Saved: COMPARE_instance1.png
Saved: COMPARE_instance2.png
Saved: COMPARE_instance3.png
Saved: COMPARE_instance4.png
Saved: COMPARE_instance5.png
Saved: COMPARE_instance6.png
Saved: COMPARE_instance7.png
Saved: COMPARE_instance8.png


In [100]:
sns.set_theme(style="whitegrid", context="notebook")
plt.rcParams['font.family'] = 'sans-serif'

METHOD_COLORS = {
    "Grasp Standard": "#1f77b4",       
    "Grasp Cost Perturb": "#ff7f0e",   
    "Grasp Intelligent": "#2ca02c",    
    "Grasp Full": "#d62728"            
}

df = pd.read_csv("ALL_RESULTS.csv")

for col in df.select_dtypes(include="object").columns:
    df[col] = df[col].str.replace("_", " ", regex=False).str.title()

instances = sorted(df["instance"].unique())
methods = sorted(df["method"].unique())

num_inst = len(instances)
cols = 2
rows = math.ceil(num_inst / cols)

fig, axes = plt.subplots(rows, cols, figsize=(20, 10), sharex=True)
axes = axes.flatten() 

lines = []
labels = []

for i, inst in enumerate(instances):
    ax = axes[i]
    dfi = df[df["instance"] == inst]
    
    for method in methods:
        dfm = dfi[dfi["method"] == method]
        
        if dfm.empty:
            continue

        color = METHOD_COLORS.get(method, "#333333")
        
        l, = ax.plot(
            dfm["iteration"],
            dfm["improved_cost"],
            color=color,
            linewidth=2,
            alpha=0.8,
            label=method
        )
        
        if i == 0:
            if method not in labels:
                labels.append(method)
                lines.append(l)

    ax.set_title(inst, fontsize=14, fontweight='bold', color='#333333')
    ax.grid(True, linestyle=':', alpha=0.6)
    sns.despine(ax=ax)
    
    if i % cols == 0:
        ax.set_ylabel("Custo", fontsize=11)
    if i >= (rows - 1) * cols:
        ax.set_xlabel("Iteração", fontsize=11)

for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

fig.legend(lines, labels, loc='upper center', ncol=4, frameon=False, fontsize=13, bbox_to_anchor=(0.5, 1.02))

plt.tight_layout()
plt.subplots_adjust(top=0.92)

filename = "ALL_INSTANCES_TRAJECTORY.png"
plt.savefig(filename, dpi=300, bbox_inches='tight')
plt.close()

print(f"Salvo com sucesso: {filename}")

Salvo com sucesso: ALL_INSTANCES_TRAJECTORY.png
